## Estimate Density from ASO 
$\rho = \frac{z}{SWE}$

In [ ]:
import datetime
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
#df= pd.read_csv("GPR_AND_ASO_ALL_FIXED.csv")


In [ ]:
# How to read the ~350 MB csv file of GPR points (2-way travel time) + ASO depths/SWE/pixel centers
dataFusion= pd.read_csv("GPR_AND_ASO_ALL_FIXED.csv")
dataFusion = gpd.GeoDataFrame(dataFusion)
dataFusion.geometry = gpd.points_from_xy(dataFusion["ASO_X"], dataFusion["ASO_Y"]) # set to GPR x,y right now
print(dataFusion.dtypes) # check that floats are floats
dataFusion.head(10000).plot() # should see spiral

In [ ]:
ASO.head()

## GPR Density and SnowPit Density

In [ ]:
## Load .csv
GPRLiDAR1 = pd.read_csv("GPRLiDAR1.csv")
snowPit1 = pd.read_csv("snow_pit_gpr1_avg.csv")
mp1 = pd.read_csv("mp_lidar1.csv")
mp1.head()

In [ ]:
## Summary Statistics
print('The GPR-LiDAR mean density is', round(GPRLiDAR1.dens.mean(),2))
print('The ASO mean density is', round(GPRLiDAR1.ASO_2020_02_02_dens.mean(),2))
print('The Snow Pit mean density is', round(snowPit1.value.mean(),2))

print('The GPR-LiDAR std density is', round(GPRLiDAR1.dens.std(),2))
print('The ASO mean std is', round(GPRLiDAR1.ASO_2020_02_02_dens.std(),2))
print('The Snow Pit std density is', round(snowPit1.value.std(),2))

print('The GPR-LiDAR min density is', round(GPRLiDAR1.dens.quantile(.01),2))
print('The ASO mean min is', round(GPRLiDAR1.ASO_2020_02_02_dens.quantile(.01),2))
print('The Snow Pit min density is', round(snowPit1.value.quantile(.01),2))

print('The GPR-LiDAR max density is', round(GPRLiDAR1.dens.quantile(.99),2))
print('The ASO mean max is', round(GPRLiDAR1.ASO_2020_02_02_dens.quantile(.99),2))
print('The Snow Pit max density is', round(snowPit1.value.quantile(.99),2))

print('The GPR-LiDAR count density is', round(GPRLiDAR1.dens.count(),2))
print('The ASO mean count is', round(GPRLiDAR1.ASO_2020_02_02_dens.count(),2))
print('The Snow Pit count density is', round(snowPit1.value.count(),2))


In [ ]:
len(snowPit1)

In [ ]:
import scipy.spatial as spatial

In [ ]:
points = np.array((snowPit1["easting"],snowPit1["northing"]))
points = points.transpose()
GPRpoints = np.array((GPRLiDAR1["ASO_X"],GPRLiDAR1["ASO_Y"]))
GPRpoints = GPRpoints.transpose()

#print(points)
#print(GPRpoints)
point_tree = spatial.cKDTree(GPRpoints)
buffer = 25 ## 25 meters
out = point_tree.query_ball_point(points, buffer)

In [ ]:
outtmp = list(filter(None,out))
len(outtmp)
len(GPRLiDAR1.dens)

In [ ]:
denseout = []
for x in out:
    denseout.append(GPRLiDAR1.iloc[x]['dens'].median())
print(len(denseout))

snowPit1['GPRdens'] = denseout

snowPit1.head(100)

In [ ]:
## Save Snow Pit Density Matches
snowPit1.to_csv('SnowPitGPRASO1.csv')

## Statistics and Plots

In [ ]:
# Calculate the Correlation ASO Density GPR LiDAR Density
import numpy.ma as ma

#A = [1, 2, 3, 4, 5, np.NaN]
#B = [2, 3, 4, 5.25, np.NaN, 100]

#print(ma.corrcoef(ma.masked_invalid(A), ma.masked_invalid(B)))
rGPRASO1 = ma.corrcoef(ma.masked_invalid(GPRLiDAR1.dens),ma.masked_invalid(GPRLiDAR1.ASO_2020_02_02_dens))
print('The correlation is', round(rGPRASO1[0,1],2))
# Calculate the Root Mean Squared Error
rmseASOgpr1 = np.sqrt(np.mean((ma.masked_invalid(GPRLiDAR1.dens)-ma.masked_invalid(GPRLiDAR1.ASO_2020_02_02_dens))**2))
print(rmseASOgpr1)

In [ ]:
# Compute the Regression Line
m, b = np. polyfit(GPRLiDAR1.dens,GPRLiDAR1.ASO_2020_02_02_dens, 1)
# Plot the Correlation
plt.figure(0)
plt.plot(GPRLiDAR1.dens,GPRLiDAR1.ASO_2020_02_02_dens,'o')
plt.plot(GPRLiDAR1.dens, m*GPRLiDAR1.dens + b,'k')
plt.xlabel('GPR-LiDAR Density [kg/m^3]')
plt.ylabel('ASO Density [kg/m^3]')

plt.figure(1)
plt.hexbin(GPRLiDAR1.dens,GPRLiDAR1.ASO_2020_02_02_dens)
plt.plot(GPRLiDAR1.dens, m*GPRLiDAR1.dens + b,'k')
plt.xlabel('GPR-LiDAR Density [kg/m^3]')
plt.ylabel('ASO Density [kg/m^3]')
plt.set_cmap('magma')
plt.savefig('ASOgprCompareDensity.png',dpi=300,bbox_inches='tight')

## Snow Pit Density and GPR LiDAR Density

In [ ]:
rPitGPR1 = ma.corrcoef(ma.masked_invalid(snowPit1.GPRdens),ma.masked_invalid(snowPit1.value))
print('The correlation is', round(rPitGPR1[0,1],2))
rmsePitgpr1 = np.sqrt(np.mean((ma.masked_invalid(snowPit1.GPRdens)-ma.masked_invalid(snowPit1.value))**2))
print(rmsePitgpr1)

In [ ]:
# Compute the Regression Line
#m, b = np. polyfit(snowPit1.GPRdens,snowPit1.value, 1)
# Plot the Correlation
plt.figure(0)
plt.plot(snowPit1.GPRdens,snowPit1.value,'o')
#plt.plot(snowPit1.GPRdens, m*snowPit1.GPRdens + b,'k')
plt.xlabel('GPR-LiDAR Density [kg/m^3]')
plt.ylabel('Snow Pit Density [kg/m]')
plt.savefig('SnowPitGprCompareDensity.png',dpi=300,bbox_inches='tight')

## Snow Pit to ASO Density

In [ ]:
## Query the KD Tree from Before (out)
denseout = []
for x in out:
    denseout.append(GPRLiDAR1.iloc[x]['ASO_2020_02_02_dens'].median())
print(len(denseout))

snowPit1['ASOdens'] = denseout



In [ ]:
rASOsnowPit1 = ma.corrcoef(ma.masked_invalid(snowPit1.ASOdens),ma.masked_invalid(snowPit1.value))
print('The correlation is', round(rASOsnowPit1[0,1],2))
rmsePitASO1 = np.sqrt(np.mean((ma.masked_invalid(snowPit1.ASOdens)-ma.masked_invalid(snowPit1.value))**2))
print(rmsePitASO1)

In [ ]:
# Compute the Regression Line
#m, b = np. polyfit(snowPit1.GPRdens,snowPit1.value, 1)
# Plot the Correlation
plt.figure(0)
plt.plot(snowPit1.ASOdens,snowPit1.value,'o')
#plt.plot(snowPit1.GPRdens, m*snowPit1.GPRdens + b,'k')
plt.xlabel('ASO Density [kg/m^3]')
plt.ylabel('Snow Pit Density [kg/m]')
plt.savefig('SnowPitASOCompareDensity.png',dpi=300,bbox_inches='tight')

## Magnaprobe Comparison

In [ ]:
points = np.array((mp1["easting"],mp1["northing"]))
points = points.transpose()
GPRpoints = np.array((GPRLiDAR1["ASO_X"],GPRLiDAR1["ASO_Y"]))
GPRpoints = GPRpoints.transpose()

print(points)
print(GPRpoints)
point_tree = spatial.cKDTree(GPRpoints)
buffer = 3 ## 5 meters
out = point_tree.query_ball_point(points, buffer)
#out = list(filter(None,out))
#len(out)

In [ ]:
depthout = []
for x in out:
    depthout.append(GPRLiDAR1.iloc[x]['ASO_2020_02_02_SD'].median())
print(len(depthout))

mp1['ASOsd'] = (depthout)
mp1['value'] = mp1.value/100

mp1.head()

In [ ]:
mp1.to_csv('ProbeASO1.csv')

In [ ]:
## Calculate Statistics
rASOmp1 = ma.corrcoef(ma.masked_invalid(mp1.ASOsd),ma.masked_invalid(mp1.value))
print('The correlation is', round(rASOmp1[0,1],2))
rmseMPASO1 = np.sqrt(np.mean((ma.masked_invalid(mp1.ASOsd)-ma.masked_invalid(mp1.value))**2))
print(rmseMPASO1)

In [ ]:
# Compute the Regression Line
#m, b = np. polyfit(snowPit1.GPRdens,snowPit1.value, 1)
# Plot the Correlation
plt.figure(0)
plt.plot(mp1.ASOsd,mp1.value,'o')
plt.plot(mp1.ASOsd, m*mp1.ASOsd + b,'k')
plt.xlabel('ASO Depth [m]')
plt.ylabel('Probe Depth [m]')
plt.savefig('ProbeASOCompareDepth.png',dpi=300,bbox_inches='tight')

In [ ]:
# Calculate the Mean Error
bias = np.mean(GPRLiDAR1.error)
print('The bias is', round(bias,2), 'cm')
# Calculate the Mean Absolute Error
mae = np.mean(np.abs(GPRLiDAR1.error))
# Calculate the Root Mean Squared Error
rmse = np.sqrt(np.mean((GPRLiDAR1.error)**2))
print('The rmse is', round(rmse,2), 'cm')
# Compute the Regression Line
m, b = np. polyfit(GPRLiDAR1.depth,GPRLiDAR1.depthGPR, 1)

# Plot the Correlation
plt.figure(0)
plt.plot(GPRLiDAR1.depth,GPRLiDAR1.depthGPR,'o')
plt.plot(GPRLiDAR1.depth, m*GPRLiDAR1.depth + b,'k')
plt.xlabel('Probe Depth [cm]')
plt.ylabel('GPR Depth [cm]')

# Plot a Histogram of the Errors
plt.figure(1)
plt.hist(GPRLiDAR1.error, density=True, bins=10, edgecolor='black')  # density=False would make counts
plt.ylabel('PDF')
plt.xlabel('Error [cm]');
# Get the Matplotlib Axes object from the dataframe object, color points by snow depth value
ax = GPRLiDAR1.plot(column='error', legend=True, cmap='PuBu')


# Use non-scientific notation for x and y ticks
ax.ticklabel_format(style='plain', useOffset=False)

# Set the various plots x/y labels and title.
ax.set_title('Error [cm] (Probed Depth - GPR Depth)')
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')